## 로지스틱 회귀

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression

```python
df = pd.read_csv('./datasets/초등_4학년_수학_차시문항풀이이력.csv', index_col=0)
pd.set_option('display.max_columns', len(df.columns))
df.head()
```

In [2]:
df = pd.read_csv('./초등_4학년_수학_차시문항풀이이력.csv', index_col=0)
pd.set_option('display.max_columns', len(df.columns))
df.head()

,사용자ID,차시코드,단원코드,단원제목,강의내용,강의명,학년,학기,시험구분,강의구분,강의타입,동영상재생시간,실제재생시간,확인문제점수,학습일,문항번호,문항코드,사용자입력,정오답,영역,대단원코드,대단원제목,중단원코드,중단원제목,소단원코드,소단원제목,토픽코드,토픽제목,난이도,평가영역
0,62fbd7b7-6a32-4da2-b203-acdec95e00b2,T0ME41U43001,T0ME41U43,1학기 복습,"만, 다섯 자리 수 알아보기",수학 1단원 【복습①】,4,1,NaN,E,AAA,0,8,-1,2022-07-04 21:00:03,1.0,30016642.0,10000,O,MA,17120995.0,1. 큰 수,14201237.0,다섯 자리 수,12233514.0,다섯 자리 수의 이해,12234054.0,모형 돈 세어 보기,2.0,91.0
1,62fbd7b7-6a32-4da2-b203-acdec95e00b2,T0ME41U43001,T0ME41U43,1학기 복습,"만, 다섯 자리 수 알아보기",수학 1단원 【복습①】,4,1,NaN,E,AAA,0,8,-1,2022-07-04 21:00:03,2.0,30016654.0,1000/^ROW^/100/^ROW^/10/^ROW^/1,O,MA,17120995.0,1. 큰 수,14201237.0,다섯 자리 수,12233514.0,다섯 자리 수의 이해,12233718.0,10000 알아보기,2.0,91.0
2,62fbd7b7-6a32-4da2-b203-acdec95e00b2,T0ME41U43001,T0ME41U43,1학기 복습,"만, 다섯 자리 수 알아보기",수학 1단원 【복습①】,4,1,NaN,E,AAA,0,8,-1,2022-07-04 21:00:03,3.0,30016589.0,713758/^ROW^/칠만십삼천칠배오십팔,X,MA,17120995.0,1. 큰 수,14201237.0,다섯 자리 수,12233514.0,다섯 자리 수의 이해,12234058.0,"10000이 ■, 1000이 ▲ ……인 다섯 자리 수를 나타내기",2.0,91.0
3,62fbd7b7-6a32-4da2-b203-acdec95e00b2,T0ME41U43001,T0ME41U43,1학기 복습,"만, 다섯 자리 수 알아보기",수학 1단원 【복습①】,4,1,NaN,E,AAA,0,8,-1,2022-07-04 21:00:03,4.0,30016571.0,60000/^ROW^/3000/^ROW^/100/^ROW^/90/^ROW^/7,O,MA,17120995.0,1. 큰 수,14201237.0,다섯 자리 수,12233515.0,다섯 자리 수의 자릿값,12233721.0,각 자릿수의 합으로 나타내기,3.0,91.0
4,62fbd7b7-6a32-4da2-b203-acdec95e00b2,T0ME41U43001,T0ME41U43,1학기 복습,"만, 다섯 자리 수 알아보기",수학 1단원 【복습①】,4,1,NaN,E,AAA,0,8,-1,2022-07-04 21:00:03,5.0,30016608.0,천/^ROW^/2000,O,MA,17120995.0,1. 큰 수,14201237.0,다섯 자리 수,12233515.0,다섯 자리 수의 자릿값,12233719.0,각 자리의 숫자와 나타내는 값 알아보기,3.0,91.0


```python
df['정오답'] =  df['정오답'].map({'O': 1, 'X': 0})
```

In [3]:
df['정오답'] =  df['정오답'].map({'O': 1, 'X': 0})

Q. 어떤 피쳐가 정오답에 영향을 미칠까?

```python
df = df[['정오답', '강의타입', '동영상재생시간', '실제재생시간', '학습일', '문항코드', '대단원코드', '중단원코드', '소단원코드', '토픽코드', '난이도', '평가영역']]
```

In [4]:
df = df[['정오답', '강의타입', '동영상재생시간', '실제재생시간', '학습일', '문항코드', '대단원코드', '중단원코드', '소단원코드', '토픽코드', '난이도', '평가영역']]

In [5]:
df = df.dropna(subset=['정오답'])

```python
df = df.dropna(subset=['정오답'])
```

In [6]:
df['학습일'] = df.학습일.apply(pd.to_datetime)
df['hour'] = df.학습일.apply(lambda x: x.hour)

```python
df['학습일'] = df.학습일.apply(pd.to_datetime)
df['hour'] = df.학습일.apply(lambda x: x.hour)
```

In [7]:
df = pd.get_dummies(df, columns=['강의타입', '문항코드', '대단원코드', '중단원코드', '소단원코드', '토픽코드', '난이도', '평가영역', 'hour'])

카테고리형 변수 원핫인코딩

```python
df = pd.get_dummies(df, columns=['강의타입', '문항코드', '대단원코드', '중단원코드', '소단원코드', '토픽코드', '난이도', '평가영역', 'hour'])
```

In [8]:
df.head(3)

,정오답,동영상재생시간,실제재생시간,학습일,강의타입_AAA,강의타입_AAB,강의타입_AAC,강의타입_AAD,강의타입_E,강의타입_GAA,강의타입_GAI,강의타입_GAJ,강의타입_GAN,강의타입_GAP,강의타입_GBA,...,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,1.0,0,8,2022-07-04 21:00:03,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1.0,0,8,2022-07-04 21:00:03,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0.0,0,8,2022-07-04 21:00:03,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


```python
df = df.drop(labels='학습일', axis=1)
```

In [9]:
df = df.drop(labels='학습일', axis=1)

In [10]:
X = df.iloc[:,1:].to_numpy()
Y = df['정오답'].to_numpy()

MemoryError: Unable to allocate 1018. MiB for an array with shape (1449, 92125) and data type int64

```python
X = df.iloc[:,1:].to_numpy()
Y = df['정오답'].to_numpy()
```

```python
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train , X_test, y_train , y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
```

```python
from sklearn.metrics import accuracy_score, roc_auc_score

# 로지스틱 회귀를 이용하여 학습 및 예측 수행. 
# solver인자값을 생성자로 입력하지 않으면 solver='lbfgs'  
lr_clf = LogisticRegression() # solver='lbfgs'
lr_clf.fit(X_train, y_train)
lr_preds = lr_clf.predict(X_test)

# accuracy와 roc_auc 측정
print('accuracy: {0:.3f}, roc_auc:{1:.3f}'.format(accuracy_score(y_test, lr_preds),
                                                 roc_auc_score(y_test , lr_preds)))
```

#### 더 성능을 높여보자.

재생시간 스케일링

```python
df[['동영상재생시간','실제재생시간']]
```

```python
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df[['동영상재생시간','실제재생시간']])
```

```python
df[['동영상재생시간','실제재생시간']] = data_scaled
```

```python
X = df.iloc[:,1:].to_numpy()
Y = df['정오답'].to_numpy()
```

```python
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train , X_test, y_train , y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
```

```python
# 로지스틱 회귀를 이용하여 학습 및 예측 수행. 
# solver인자값을 생성자로 입력하지 않으면 solver='lbfgs'  
lr_clf = LogisticRegression() # solver='lbfgs'
lr_clf.fit(X_train, y_train)
lr_preds = lr_clf.predict(X_test)

# accuracy와 roc_auc 측정
print('accuracy: {0:.3f}, roc_auc:{1:.3f}'.format(accuracy_score(y_test, lr_preds),
                                                 roc_auc_score(y_test , lr_preds)))
```

```python
pd.DataFrame(data=[lr_preds, y_test])
```